In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [2]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [4]:
design.chips['C_chip'].size['size_x']='3mm'
design.chips['C_chip'].size['size_y']='3mm'
design.chips['Q_chip'].size['size_x']='2mm'
design.chips['Q_chip'].size['size_y']='2mm'
design.chips['Q_chip'].size['center_z']='8um'

In [5]:
design.get_chip_z('Q_chip')

'8um'

In [6]:

cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-300um',
                                            pos_y='-500um',
                                            chip='Q_chip',
                                            pad_width='370um',
                                            pad_height='100um',
                                            pad_gap='80um',
                                            pocket_width='680um',
                                            pocket_height='680um'))]
                                            
qubits=q0
q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = StraightUpCoupler(design, "c0", options=dict(pos_x=f"{q0x-120}um",
                                              pos_y=f"{q0y+90}um",
                                              pad_width="165um",
                                              pad_height="100um",
                                              cpw_stub_height="200um",
                                              chip="C_chip"))
design.rebuild()
gui.rebuild()
gui.autoscale()

In [7]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="C_chip",
                                         pos_x = '-700um',
                                         pos_y = '800um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="C_chip",
                                         pos_x = '700um',
                                         pos_y = '800um',
                                         orientation='180',
                                         lead_length = '30um'))                                        

In [8]:
stg=ShortToGround(design, 'stg', options=Dict(chip="C_chip",
                                                pos_x='0.65mm',
                                                pos_y='-0.2mm',
                                                orientation='270'))

In [9]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [10]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

# Resonator length
3.85mm

In [11]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [12]:
resonator_length_4=[]
a=find_resonator_length(7.5,10,6,4)
a=a.strip('mm')
a=float(a)-0.25 +0.5-0.23-0.5# subtract coupler length, for +0.5Ghz, coupler length
resonator_length_4.append(a)
print(resonator_length_4)


[3.5775622473488387]


In [13]:
jogs1=OrderedDict()
jogs1[0]=['R','300um']
jogs1[1]=['R', '300um']
jogs1[2]=['L','700um']
jogs1[3]=['R','200um']
jogs1[4]=['R','670um']
jogs1[5]=['L','200um']
jogs1[6]=['L','740um']


In [14]:
rr0=connect('rr0', 'c0', 'in','stg', 'short',
            resonator_length_4[0],jogs1,jogs, 
            '100um','0um', '944um', '100um')
            

# EPR Analysis

In [15]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [16]:
hfss.start()

INFO 05:43PM [connect_project]: Connecting to Ansys Desktop API...
INFO 05:43PM [load_ansys_project]: 	Opened Ansys App
INFO 05:43PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 05:43PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   Flipchip Bump
INFO 05:43PM [connect_design]: 	Opened active design
	Design:    flip6um_V7 [Solution type: Eigenmode]
INFO 05:43PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 05:43PM [connect]: 	Connected to project "Flipchip Bump" and design "flip6um_V7" 😀 



True

In [17]:
hfss.activate_ansys_design("flip8um_V7", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','stg','rr0','fl0'],[],
                    box_plus_buffer=False)

05:43PM 30s WARNING [activate_ansys_design]: The design_name=flip8um_V7 was not in active project.  Designs in active project are: 
['flip7um_BUMP', 'flip6um_V7', 'flip7um_V6', 'flip8um_V6_BUMP', 'flip6um_V6', 'flip7um_V7', 'flip6um_BUMP', 'flip8um_V6'].  A new design will be added to the project.  
INFO 05:43PM [connect_design]: 	Opened active design
	Design:    flip8um_V7 [Solution type: Eigenmode]
WARNING 05:43PM [connect_setup]: 	No design setup detected.
WARNING 05:43PM [connect_setup]: 	Creating eigenmode default setup.
INFO 05:43PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [20]:
hfss.close()


True

# LOM Analysis

In [14]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [15]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='5.82',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '5.82',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [19]:
#c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 05:23PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp0ddd0sy8.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5820000000, Maxwell, 1, False
INFO 05:23PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp5r6a8_hg.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5820000000, Maxwell, 1, False
INFO 05:23PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpsybm71yf.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5820000000, Maxwell, 2, False
INFO 05:23PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp63t5eb2p.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5820000000, Maxwell, 3, False
INFO 05:23PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpqyspbx_3.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5820000000, Maxwell, 4, False
INFO 05:23

,pad_bot_Q0,pad_top_Q0,quad_coupler_c0,SignalNet1
pad_bot_Q0,130.02000,-11.941,-0.10109,-116.640
pad_top_Q0,-11.94100,129.620,-22.41000,-93.896
quad_coupler_c0,-0.10109,-22.410,114.30000,-91.931
SignalNet1,-116.64000,-93.896,-91.93100,366.810


In [18]:
c1.setup.junctions = Dict({'Lj': 12, 'Cj': 2})
c1.setup.freq_readout = 7.02
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

NameError: name 'c1' is not defined

In [22]:
c1.sim.close()

Warning! 3 COM references still alive
Ansys will likely refuse to shut down
